In [315]:
# %pip install git+https://github.com/tensorflow/docs

import os.path
from typing import List

import numpy as np
import pandas as pd

from datetime import date, datetime, timedelta
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


# Config

In [316]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344 Version 12.13.453.3037 (Jul 11 2022/18:39:20) [PUBLIC] <Releases/12.13>
RANKED_ID = 1100    # 1090 normal game 1100 ranked game
PATCH: date = date(2022, 7, 1)  # date(2022, 7, 1)

TARGETNAME = 'placement'

In [317]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [318]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1nGYZSogOi_2_zhf9qMFcw0z7-O7oLphGUETauYDmbHoVS3n,5I3IY-0ZKUSrw12aj63qHxH5bP2sDmVfwElhaZpMstBmVjM-ceSag-gV,2_VlG67OPILEx6raP4Iooh0v4XWnVjnfvmDXx0bm7KbJ2MEOmundLJSLHZctwuh2b_AMEud8kaOCxQ,yolomycolin,1588,1657499136000,146,1nGYZSogOi_2_zhf9qMFcw0z7-O7oLphGUETauYDmbHoVS3n,yolomycolin,253,I,152,122,False,False,False,False
1,QJYrWen8TNiR1wJNv5Lr5kFkdUJ6myMADxvMsTfcQ3CBdl5_,PffmeMHAwzu5DsAVS1HaLCLnDh-TEExa-RcfcEAjCN4hvYF8l1kL9vOF,x5LHxVz73kwoJI-Cup6itRsBMuUgLP6Bh8DcP2PARaGSXk_cnvZnpoGDAL4S4FFKYmur0AsmYPaWew,LIMIT TEST,4568,1657702512000,36,QJYrWen8TNiR1wJNv5Lr5kFkdUJ6myMADxvMsTfcQ3CBdl5_,LIMIT TEST,320,I,54,23,False,False,True,False
2,iAvqT_I1QMSvRinhz-qp4ogFZ-yftPtD3q74jXf9AdU,02I1QCLMWMDfo-nHUmUgulJo4jnnx1qiUbJOkK4PnKg,_NfGsxwwohzDv11wiKeUwOUx5tG6AmMLx5jDmMZ0YInV394Z4b_jPuGBYNM9HrCzJC16ZH2-k8HvDg,Freebìrd,575,1657686286000,187,iAvqT_I1QMSvRinhz-qp4ogFZ-yftPtD3q74jXf9AdU,Freebìrd,283,I,108,107,False,False,False,False
3,wOl5iKlDAermNYtwN41ftRN0qFzXNNCAMZlt9VRqAy06o9c,lCBGXPoEAFml-rvkCmvo4EDbFOv8Wp_SBvYsfBG5yedN1A,20HcCpfGQ5v2Zhi-mX465x4WW0Lu55Xp1PRSiLO5-8Tc4jvni-Xp9XOvb4aSpJI57OagjEgepyy0tw,Rolling ZoZo,588,1657687757000,157,wOl5iKlDAermNYtwN41ftRN0qFzXNNCAMZlt9VRqAy06o9c,Rolling ZoZo,294,I,119,92,False,False,True,False
4,ZiwNT-uQ9h74CVosHNwnN3o0uluYlnANTJgVHWcBLIuLIsU,9Hef8ErlbaK4AeWHAAZU3QTmkrHv1vvfTZP1Di-S0JQU1ak,dgVywTR_8rqCRmaZZG87wyX0rOEnGLAFSEmL9Bs56UwERdgzXEyakf_3Zyws_2nK9UQ48rghuxqX4A,Vitamnss,3901,1657658351000,117,ZiwNT-uQ9h74CVosHNwnN3o0uluYlnANTJgVHWcBLIuLIsU,Vitamnss,356,I,93,65,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,cgJs7YT1KMdTcjADb_jZmHlrt62s9PE7cC0GeNysq9Gwc-tK,y2cKMns3KSslMFWNiRf5_jk14tSDPk9mTqWfpnxrh3GAbAHmFLU8Gtip,th7Fki8t0HH2wDVvrbHq-6IuS1O9OHeeXlpRG2_S_TdGRnYeOdM6jOHVSKczOtc0O-i0O5BcP93wWA,Snoissyba,4025,1657566137000,51,cgJs7YT1KMdTcjADb_jZmHlrt62s9PE7cC0GeNysq9Gwc-tK,Snoissyba,388,I,103,68,False,False,False,False
496,Y238X-_ex4QrHoZyfC0WMDRIEPaMf3hYu3mtarPxs8HGsgU,DV8Ia9UklHCl2uW1T3akkfnGZbQmt8rIoyJg0ZxO11IA5g,mbWc6enyl7n6lezfu0aylpotPXk8E3w4kb3MioOPLDXWO4WMyaj2UjyDlirukhK04DWCPul76YMufA,EmperorOng,5250,1657701719199,160,Y238X-_ex4QrHoZyfC0WMDRIEPaMf3hYu3mtarPxs8HGsgU,EmperorOng,253,I,102,79,False,False,True,True
497,nXHVyc6n6oM8FZB8-fowWO1Zs1TZqBNfCLwmFj6njcEN05k,JZlKGV5pD2kP9grNUjU_LdrBZcoQLaqkgaP1NiyE9vTGyGg,pEx-MN8-ToY-V1ePrfZBiJlJa1Tp4FHJ5IBB-nhOxHO1-cmtZLWpFiI8s6mgRVwCtxtx_9cEZU-Ptg,Bipoc Shakur,3602,1657700618000,345,nXHVyc6n6oM8FZB8-fowWO1Zs1TZqBNfCLwmFj6njcEN05k,Bipoc Shakur,323,I,104,84,False,False,True,True
498,nZplmLu2CEwaxagAEFUmGrWvE5d7r8VVhH1LSueF9Z5tUrs,ArTUmQatfBEWQg-KCXWFSJ4O6rGNVCz4pko1F8TWwqSEGg,6jk_MlXODLfKm38N-byj1W-9XT2GUHhR0231CbKrLPp0N92Moge430ox1ajqVva5tIv53zcTUFfeBg,Myman580,4962,1657686311000,287,nZplmLu2CEwaxagAEFUmGrWvE5d7r8VVhH1LSueF9Z5tUrs,Myman580,233,I,105,80,False,False,True,False


# Load unique matches id

In [319]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [320]:
len(uniq_matches_id)

16242

In [321]:
len(matches_asset)

18274

In [322]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [323]:
len(uniq_matches)

16242

## Filter by LATEST_RELEASE version and RANKED_ID game.

In [324]:
matches_asset[-1]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [325]:
matches_asset[-1]['info']['queue_id']

1100

In [326]:
latest_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version']) and (
    RANKED_ID == match['info']['queue_id'])]


In [327]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [328]:
timedelta(days=3)

datetime.timedelta(days=3)

In [329]:
latest_3d_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                     and ((datetime.now() - timedelta(days=3)) <= datetime.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [330]:
len(latest_matches)

14125

In [331]:
len(latest_patch_matches)

12458

In [332]:
len(latest_3d_matches)

3195

# Process api details to datasets rows

In [333]:
def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [334]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)
matches_3d_array = process_matches(latest_3d_matches)

In [335]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4360319488',
 'placement': 5,
 'augment0': 'TFT6_Augment_Ascension',
 'augment1': 'TFT6_Augment_ItemGrabBag1',
 'augment2': 'TFT7_Augment_BruiserTitanicStrength',
 'Set7_Assassin': 1,
 'Set7_Bard': 1,
 'Set7_Bruiser': 2,
 'Set7_Dragon': 1,
 'Set7_Guardian': 0,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Mage': 0,
 'Set7_Mystic': 0,
 'Set7_Ragewing': 0,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'Set7_Whispers': 3,
 'TFT7_Thresh': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Qiyana': 1,
 'TFT7_Shen': 2,
 'TFT7_Shen_item0': 'RedBuff',
 'TFT7_Shen_item1': 'ZekesHerald',
 'TFT7_Sylas': 2,
 'TFT7_DragonPurple': 2,
 'TFT7_DragonPurple_item0': 'TitansResolve',
 'TFT7_DragonPurple_item1': 'UnstableConcoction',
 'TFT7_DragonPurple_item2': 'Quicksilver',
 'TFT7_Ornn': 2,
 'TFT7_Ornn_item0': 'RedBuff',
 'TFT7_Bard': 2,
 'TFT7_Bard_item0': 'GuinsoosRageblade',
 'TFT7_Pyke': 1,
 'TFT7_Pyke_item0': 'Bloodthirster',
 'TFT7_Pyke_item1': 'WarmogsAr

In [336]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)
matches_league_3d_df = pd.json_normalize(matches_3d_array)

In [337]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bard,Set7_Bruiser,Set7_Dragon,Set7_Guardian,...,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Sejuani_item2,TFT7_Senna,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4360319488,5,TFT6_Augment_Ascension,TFT6_Augment_ItemGrabBag1,TFT7_Augment_BruiserTitanicStrength,1.0,1.0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4360319488,6,TFT6_Augment_CalculatedLoss,TFT6_Augment_Weakspot,TFT6_Augment_PortableForge,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4360319488,3,TFT6_Augment_Ascension,TFT7_Augment_RagewingTrait,TFT6_Augment_PortableForge,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4360319488,4,TFT7_Augment_ShimmerscaleTrait,TFT6_Augment_CelestialBlessing1,TFT6_Augment_CyberneticShell2,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4360319488,8,TFT6_Augment_TrueTwos,TFT6_Augment_Diversify1,TFT7_Augment_UrfsGrabBag1,2.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112995,NA1_4364962841,8,TFT6_Augment_TinyTitans,TFT6_Augment_SecondWind2,TFT6_Augment_Diversify2,0.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112996,NA1_4364962841,2,TFT7_Augment_BandOfThieves1,TFT7_Augment_Preparation2,TFT6_Augment_PortableForge,NaN,NaN,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112997,NA1_4364962841,7,TFT6_Augment_SecondWind1,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_BigFriend2,1.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112998,NA1_4364962841,3,TFT6_Augment_Weakspot,TFT7_Augment_GadgetExpert,TFT6_Augment_CyberneticUplink2,0.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [338]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bard,Set7_Bruiser,Set7_Dragon,Set7_Guardian,...,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Sejuani_item2,TFT7_Senna,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4360319488,5,TFT6_Augment_Ascension,TFT6_Augment_ItemGrabBag1,TFT7_Augment_BruiserTitanicStrength,1.0,1.0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4360319488,6,TFT6_Augment_CalculatedLoss,TFT6_Augment_Weakspot,TFT6_Augment_PortableForge,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4360319488,3,TFT6_Augment_Ascension,TFT7_Augment_RagewingTrait,TFT6_Augment_PortableForge,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4360319488,4,TFT7_Augment_ShimmerscaleTrait,TFT6_Augment_CelestialBlessing1,TFT6_Augment_CyberneticShell2,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4360319488,8,TFT6_Augment_TrueTwos,TFT6_Augment_Diversify1,TFT7_Augment_UrfsGrabBag1,2.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112995,NA1_4364962841,8,TFT6_Augment_TinyTitans,TFT6_Augment_SecondWind2,TFT6_Augment_Diversify2,0.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112996,NA1_4364962841,2,TFT7_Augment_BandOfThieves1,TFT7_Augment_Preparation2,TFT6_Augment_PortableForge,NaN,NaN,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112997,NA1_4364962841,7,TFT6_Augment_SecondWind1,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_BigFriend2,1.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112998,NA1_4364962841,3,TFT6_Augment_Weakspot,TFT7_Augment_GadgetExpert,TFT6_Augment_CyberneticUplink2,0.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [339]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [340]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)
matches_league_3d_df = reorder_df_col(matches_league_3d_df)

# Output dataframes

In [341]:
# matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
# matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)
matches_league_3d_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.pickle'))
matches_league_3d_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.csv'), index=False)

# End